# Setup Kubernetes environment for DYNAMOS in FABRIC 

This Jupyter notebook will create the Kubernetes environment in FABRIC after the slice and corresponding nodes have been created.

FABRIC API docs: https://fabric-fablib.readthedocs.io/en/latest/index.html


## Step 1:  Configure the Environment & Create Slice

Before running this notebook, you will need to configure your environment using the [Configure Environment](../configure_and_validate.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

After following all steps of the Configuring Environment notebook, you should be able to run this notebook without additional steps.

Next, you will need to have setup the slice in FABRIC using the [Create Slice](../create_slice.ipynb) notebook.

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).
 

## Step 2: Setup the Environment for this Notebook

### Step 2.1: Import FABRIC API and other libraries

In [ ]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();


### (Optional): Query for Available Testbed Resources and Settings

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity and other settings or available resources.

In [ ]:
# Comment out lists you do not want to view

# List available sites
# fablib.list_sites()
# List available hosts
# fablib.list_hosts(includes=["CERN", "AMST"])
# List available images that can be used for the nodes
fablib.get_image_names()

### Step 2.2: Configure the parameters, get Attributes, and add the nodes
Can be used to set the corresponding slice and other variables used for subsequent cells.

In [ ]:
# ========== Step 1: Configure Parameters ==========
slice_name = 'DYNAMOS'
# Set image to ubuntu specific (used in FABRIC Kubernetes cluster example)
image = "default_ubuntu_20"
# Site used for the nodes (Use the UvA site: AMST)
site = "AMST"
# Host of the site used for the nodes (Use specific host to avoid error occurred without it: "Timeout waiting for the server to come up")
site_host = "amst-w1.fabric-testbed.net"
# Nodes:
node1_name = 'Node1'
node2_name = 'Node2'
# TODO: more variables and below needed?
node1_nic_name = 'NIC1'
node2_nic_name = 'NIC2'
network_name = 'NET1'

# ========== Step 2: Get and Print Node Attributes ==========
try:
    # Get slice by name: https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.get_slice
    slice = fablib.get_slice(name=slice_name)

    # Add Nodes with the specific variables
    # Also validate the node and raise an exception in case of failure
    node1 = slice.add_node(name=node1_name, site=site, validate=True, raise_exception=True, host=site_host, image=image)
    node2 = slice.add_node(name=node2_name, site=site, validate=True, raise_exception=True, host=site_host, image=image)

    # Submit the updates. Wait until the state is finished and use an interval and a longer timeout to make sure all nodes are created
    # Progress shows an indicator of the current progression.
    slice.submit(wait=True, wait_timeout=3600, wait_interval=20, progress=True, wait_jupyter='text')
    # # Get slice nodes
    # for node in slice.get_nodes():
    #     print(f"{node}")
    # TODO: set nodes here as well
    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

### Step 2.3: Observe the Slice's Node Attributes
This step verifies the creation of the nodes.

#### List the nodes

In [ ]:
slice.list_nodes();

#### Run a test Experiment to Validate the nodes

Most experiments will require automated configuration and execution. You can use the fablib library to execute arbitrary commands on your node. 

The following code demonstrates how to use fablib to execute a "Hello, FABRIC" bash script. The library uses the bastion and VM keys defined at the top of this notebook to jump through the bastion host and execute the script.

In [ ]:
#node = slice.get_node('Node1')

for node in slice.get_nodes():
    stdout, stderr = node.execute('echo Hello, FABRIC from node `hostname -s`')

## Step 3: Configure Kubernetes cluster

This step configures the Kubernetes cluster using scripts in this folder.

### Configure Control Plane node
TODO: also configure worker node.

In [ ]:
try:
    # TODO: now also maybe some other steps need to happen before this, and the scripts need to be created to configure the control plane and worker, etc.
    # Upload script file to the nodes
    file_attributes = node1.upload_file(local_file_path="config_control_plane.sh", remote_file_path="config_control_plane.sh")
    # Execute the script
    stdout, stderr = node1.execute(f"chmod +x config_control_plane.sh && ./config_control_plane.sh")

except Exception as e:
    print(f"Exception: {e}")